In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
shark_attack = pd.read_csv('attacks.csv', encoding='latin_1')

In [3]:
#strip spaces from column names (ie "Sex ")
shark_attack.columns = map(str.strip,shark_attack.columns)

In [4]:
#Remove rows and columns that are not needed
#remove rows where the Date/Year is N/A, I've prioritized knowing when the attack happened as being important
shark_attack.dropna(subset=['Date','Year'], inplace=True)

#Column Titles unclear and 99.9% of the data is null so they are dropped
shark_attack.drop(columns=['Unnamed: 22', 'Unnamed: 23'], inplace=True)

#href formula and href contain duplicate information. href has no null values so keeping href. Case Number.1 and Case Number.2 are duplicates of Case Number
shark_attack.drop(columns=['href formula','Case Number.1','Case Number.2'], inplace=True)

#"pdf" and "original order"  have no aded value, so bye-bye
shark_attack.drop(columns=['pdf','original order'], inplace=True)

In [5]:
#Clean up date column, only keep text in the format dd-MMM-yyyy
shark_attack['Date'] = shark_attack['Date'].str.extract(r'\b(\d{2}-[a-zA-z]{3}-\d{4})\b', expand=False)

# clean up time formatting, extract time in HHhmm format:
shark_attack['Time'] = shark_attack['Time'].str.extract(r'\b(\d{2}h\d{2})\b', expand=False)

In [6]:
# Fatal (Y/N) column has mix of true/false, yes, no, other data.  Standardize on Fatal Attack = 1, Non-Fatal Attack = 0, UNKNOWN =-1
shark_attack.replace({'Fatal (Y/N)': r'\b(Y|y|True)\b'}, {'Fatal (Y/N)':1}, regex=True,inplace= True)
shark_attack.replace({'Fatal (Y/N)': r'\b(N|n|False)\b'}, {'Fatal (Y/N)':0}, regex=True,inplace= True)
shark_attack.replace({'Fatal (Y/N)': r'^[^01].*$'}, {'Fatal (Y/N)':-1}, regex=True,inplace= True)
shark_attack.replace({'Fatal (Y/N)': ''}, {'Fatal (Y/N)':-1}, inplace= True)
shark_attack['Fatal (Y/N)'].fillna(-1, inplace=True)

In [7]:
#Clean up Sex column
shark_attack.replace({'Sex': r'\b(M|m)\b'}, {'Sex':'M'}, regex=True,inplace= True)
shark_attack.replace({'Sex': r'\b(F|f)\b'}, {'Sex':'F'}, regex=True,inplace= True)
shark_attack.replace({'Sex': r'^[^MF].*$'}, {'Sex':''}, regex=True,inplace= True)

In [8]:
#update data_types
shark_attack['Year'] = shark_attack['Year'].astype('int64')
shark_attack['Date'] = pd.to_datetime(shark_attack['Date'], errors="coerce") # there was an attack in 1580 which is less than the 1677 allowed by pandas
shark_attack['Time'] = pd.to_datetime(shark_attack['Time'], errors="coerce",format = '%Hh%M').dt.time

In [9]:
shark_attack.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'Investigator or Source', 'href'],
      dtype='object')

In [13]:
#rename columns
shark_attack = shark_attack.rename(columns={'Sex': 'Gender', 'Fatal (Y/N)': 'Fatal'})

In [14]:
#write out clean file to csv
shark_attack.to_csv('shark_attack.csv',index=False)